In [1]:
import ast
import os
import random
import warnings

import autogluon.tabular
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection

warnings.filterwarnings("ignore", message="Can't initialize NVML")

In [2]:
DIRECTORY_DATA = 'data/model-autogluon'
os.makedirs(DIRECTORY_DATA, exist_ok=True)

In [3]:
COLUMNS_NONSPLIT = {
    'Scenario ID': lambda series: series.values,
}
COLUMNS_INPUT = {
    'Coordination strategy': lambda series: series.astype('category').cat.codes.values,
    'isCanPassFirstHum': lambda series: series.astype('int').values,
    'isRacingThroughCrossroadAllowed': lambda series: series.astype('int').values,
    
    'Vehicle ID': lambda series: series.values,
    'Linearization C': lambda series: [
        series.apply(lambda x: -1 if x is None else x[i])
        for i in range(len(series.dropna().iloc[0]))
    ],   
}
COLUMNS_OUTPUT = {
    'traveled total, m': lambda series: series.values,
    'No. of completed missions': lambda series: series.values,
    'No. of collisions': lambda series: series.values,
    'No. of near-misses': lambda series: series.values,
}

COLUMNS_ALL = {**COLUMNS_NONSPLIT, **COLUMNS_INPUT, **COLUMNS_OUTPUT}

In [4]:
runnames = [
    '20241203_170129_all600',
    '20241213_104400_racing',
    '20241214_122216_racing_passhum',
]
df_all = pd.concat(
    [pd.read_csv(f'data/{runname}/df_all.csv') 
     for runname in runnames],
    keys=runnames,
    names=['runname', 'row']
)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7176 entries, ('20241203_170129_all600', 0) to ('20241214_122216_racing_passhum', 2383)
Data columns (total 83 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         7176 non-null   int64  
 1   i_map                              7176 non-null   int64  
 2   are_bridges                        7176 non-null   bool   
 3   Positions variant                  7176 non-null   int64  
 4   configuration                      7176 non-null   object 
 5   Coordination strategy              7176 non-null   object 
 6   probabilityForcingForHuman         7176 non-null   float64
 7   filename_screenshot                7176 non-null   object 
 8   isCanPassFirstHum                  7176 non-null   bool   
 9   isCanPassFirstAut                  7176 non-null   bool   
 10  Date                               7176 non-null   objec

In [16]:
def do_temporary_filtering(df):
    df = df[df.index.get_level_values('runname') == '20241203_170129_all600']
    df = df[df['Coordination strategy'] == 'baseline']
    df = df[df['No. of collisions'] != 0]
    return df


do_temporary_filtering(df_all)

Unnamed: 0  i_map  are_bridges  \
runname                row                                    
20241203_170129_all600 961          961      5         True   
                       970          970      5         True   
                       973          973      5         True   
                       982          982      5         True   
                       985          985      5         True   
...                                 ...    ...          ...   
                       1653        1653      7        False   
                       1657        1657      7        False   
                       1666        1666      7        False   
                       1668        1668      7        False   
                       1677        1677      7        False   

                             Positions variant  \
runname                row                       
20241203_170129_all600 961                   1   
                       970                   1   
                       973                   2   
                       982                   2   
                       985                   3   
...                                        ...   
                       1653                  8   
                       1657                  9   
                       1666                  9   
                       1668                 10   
                       1677                 10   

                                                   configuration  \
runname                row                                         
20241203_170129_all600 961       map 5, with bridges, pos.var. 1   
                       970       map 5, with bridges, pos.var. 1   
                       973       map 5, with bridges, pos.var. 2   
                       982       map 5, with bridges, pos.var. 2   
                       985       map 5, with bridges, pos.var. 3   
...                                                          ...   
                       1653   map 7, without bridges, pos.var. 8   
                       1657   map 7, without bridges, pos.var. 9   
                       1666   map 7, without bridges, pos.var. 9   
                       1668  map 7, without bridges, pos.var. 10   
                       1677  map 7, without bridges, pos.var. 10   

                            Coordination strategy  probabilityForcingForHuman  \
runname                row                                                      
20241203_170129_all600 961               baseline                         0.0   
                       970               baseline                         0.0   
                       973               baseline                         0.0   
                       982               baseline                         0.0   
                       985               baseline                         0.0   
...                                           ...                         ...   
                       1653              baseline                         0.0   
                       1657              baseline                         0.0   
                       1666              baseline                         0.0   
                       1668              baseline                         0.0   
                       1677              baseline                         0.0   

                                                           filename_screenshot  \
runname                row                                                       
20241203_170129_all600 961   ../map-generator/generated-maps/2024-11-28_13:...   
                       970   ../map-generator/generated-maps/2024-11-28_13:...   
                       973   ../map-generator/generated-maps/2024-11-28_13:...   
                       982   ../map-generator/generated-maps/2024-11-28_13:...   
                       985   ../map-generator/generated-maps/2024-11-28_13:...   
...                                          

In [5]:
df_inout = df_all[list(COLUMNS_ALL)]
df_inout

Scenario ID  \
runname                        row                                                       
20241203_170129_all600         0     map-generator/generated-maps/2024-11-28_13:17:...   
                               1     map-generator/generated-maps/2024-11-28_13:17:...   
                               2     map-generator/generated-maps/2024-11-28_13:17:...   
                               3     map-generator/generated-maps/2024-11-28_13:17:...   
                               4     map-generator/generated-maps/2024-11-28_13:17:...   
...                                                                                ...   
20241214_122216_racing_passhum 2379  map-generator/generated-maps/2024-11-28_13:19:...   
                               2380  map-generator/generated-maps/2024-11-28_13:19:...   
                               2381  map-generator/generated-maps/2024-11-28_13:19:...   
                               2382  map-generator/generated-maps/2024-11-28_13:19:...   
                               2383  map-generator/generated-maps/2024-11-28_13:19:...   

                                    Coordination strategy  isCanPassFirstHum  \
runname                        row                                             
20241203_170129_all600         0                    stops              False   
                               1     change of priorities              False   
                               2                 baseline              False   
                               3                    stops              False   
                               4     change of priorities              False   
...                                                   ...                ...   
20241214_122216_racing_passhum 2379  change of priorities               True   
                               2380                 stops               True   
                               2381              baseline               True   
                               2382  change of priorities               True   
                               2383                 stops               True   

                                     isRacingThroughCrossroadAllowed  \
runname                        row                                     
20241203_170129_all600         0                               False   
                               1                               False   
                               2                               False   
                               3                               False   
                               4                               False   
...                                                              ...   
20241214_122216_racing_passhum 2379                             True   
                               2380                             True   
                               2381                             True   
                               2382                             True   
                               2383                             True   

                                     Vehicle ID  \
runname                        row                
20241203_170129_all600         0              0   
                               1              0   
                               2              0   
                               3              1   
                               4              1   
...                                         ...   
20241214_122216_racing_passhum 2379           2   
                               2380           2   
                               2381           3   
                               2382           3   
                               2383           3   

                                                                       Linearization C  \
runname                        row                                                       
20241203_170129_all600         0     (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [6]:
def parse_tuple_string(tuple_string):
    if pd.isna(tuple_string):
        return None
    return ast.literal_eval(tuple_string)


def preprocess_inout(df_inout):
    df_inout = df_inout.copy()
    
    # Find the column that starts with 'Linearization'
    linearization_columns = [col for col in df_inout.columns if col.startswith('Linearization')]
    for col in linearization_columns:
        df_inout[col] = df_inout[col].apply(parse_tuple_string)
        
    dict_preprocessed = {}
    for col, series2data in COLUMNS_ALL.items():
        data = series2data(df_inout[col])
        if not isinstance(data, list):
            dict_preprocessed[col] = data
        else:
            for i, series in enumerate(data):
                dict_preprocessed[f'{col}#{i}'] = series
    
    return pd.DataFrame(dict_preprocessed)
    

df_preprocessed = preprocess_inout(df_inout)
df_preprocessed

Scenario ID  \
runname                        row                                                       
20241203_170129_all600         0     map-generator/generated-maps/2024-11-28_13:17:...   
                               1     map-generator/generated-maps/2024-11-28_13:17:...   
                               2     map-generator/generated-maps/2024-11-28_13:17:...   
                               3     map-generator/generated-maps/2024-11-28_13:17:...   
                               4     map-generator/generated-maps/2024-11-28_13:17:...   
...                                                                                ...   
20241214_122216_racing_passhum 2379  map-generator/generated-maps/2024-11-28_13:19:...   
                               2380  map-generator/generated-maps/2024-11-28_13:19:...   
                               2381  map-generator/generated-maps/2024-11-28_13:19:...   
                               2382  map-generator/generated-maps/2024-11-28_13:19:...   
                               2383  map-generator/generated-maps/2024-11-28_13:19:...   

                                     Coordination strategy  isCanPassFirstHum  \
runname                        row                                              
20241203_170129_all600         0                         2                  0   
                               1                         1                  0   
                               2                         0                  0   
                               3                         2                  0   
                               4                         1                  0   
...                                                    ...                ...   
20241214_122216_racing_passhum 2379                      1                  1   
                               2380                      2                  1   
                               2381                      0                  1   
                               2382                      1                  1   
                               2383                      2                  1   

                                     isRacingThroughCrossroadAllowed  \
runname                        row                                     
20241203_170129_all600         0                                   0   
                               1                                   0   
                               2                                   0   
                               3                                   0   
                               4                                   0   
...                                                              ...   
20241214_122216_racing_passhum 2379                                1   
                               2380                                1   
                               2381                                1   
                               2382                                1   
                               2383                                1   

                                     Vehicle ID  Linearization C#0  \
runname                        row                                   
20241203_170129_all600         0              0                0.0   
                               1              0                0.0   
                               2              0                0.0   
                               3              1                0.0   
                               4              1                0.0   
...                                         ...                ...   
20241214_122216_racing_passhum 2379           2                0.0   
                               2380           2                0.0   
                               2381           3                0.0   
                               2382           3                0.0   
                               2383           3                0.0   

  

In [7]:
def show(obj, title=None):
    if title is not None:
        display(HTML(f"<h3>{title}</h3>"))
    display(obj)


def shuffle_df(df):
    # Shuffle by Scenario ID
    unique_scenarios = df['Scenario ID'].unique()  # Get unique Scenario IDs
    shuffled_scenarios = pd.Series(unique_scenarios).sample(frac=1, random_state=1).tolist()  # Shuffle Scenario IDs
    
    # Reorder the dataframe by the shuffled Scenario IDs
    df_shuffled = pd.concat([df[df['Scenario ID'] == scenario] for scenario in shuffled_scenarios])
    
    # Reset index (optional)
    df_shuffled = df_shuffled.reset_index(drop=True)
    
    return df_shuffled


def split_train_test(df):
    # Define groups based on 'Scenario ID'
    groups = df['Scenario ID']
    
    # Create GroupShuffleSplit instance
    gss = sklearn.model_selection.GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
    
    # Split the data
    for train_idx, test_idx in gss.split(df, groups=groups):
        return shuffle_df(df.iloc[train_idx]), shuffle_df(df.iloc[test_idx])


df_train, df_test = split_train_test(df_preprocessed)
show(df_train, 'df_train')
show(df_test, 'df_test')

,Scenario ID,Coordination strategy,isCanPassFirstHum,isRacingThroughCrossroadAllowed,Vehicle ID,Linearization C#0,Linearization C#1,Linearization C#2,Linearization C#3,Linearization C#4,...,Linearization C#94,Linearization C#95,Linearization C#96,Linearization C#97,Linearization C#98,Linearization C#99,"traveled total, m",No. of completed missions,No. of collisions,No. of near-misses
0,map-generator/generated-maps/2024-11-28_13:17:...,2,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.746372,0.746372,0.746372,0.746372,0.82686,0.82686,75.5,0,0,0
1,map-generator/generated-maps/2024-11-28_13:17:...,2,0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,9096.5,11,0,0
2,map-generator/generated-maps/2024-11-28_13:17:...,2,0,0,2,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,9460.0,6,0,0
3,map-generator/generated-maps/2024-11-28_13:17:...,2,0,0,3,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,80.7,0,0,0
4,map-generator/generated-maps/2024-11-28_13:17:...,2,0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.746372,0.746372,0.746372,0.746372,0.82686,0.82686,75.5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5731,map-generator/generated-maps/2024-11-28_13:19:...,2,0,1,3,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,7340.8,6,1,0
5732,map-generator/generated-maps/2024-11-28_13:19:...,2,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,9002.3,11,7,1
5733,map-generator/generated-maps/2024-11-28_13:19:...,2,1,1,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,8637.3,14,0,0
5734,map-generator/generated-maps/2024-11-28_13:19:...,2,1,1,2,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,8374.3,8,2,1


,Scenario ID,Coordination strategy,isCanPassFirstHum,isRacingThroughCrossroadAllowed,Vehicle ID,Linearization C#0,Linearization C#1,Linearization C#2,Linearization C#3,Linearization C#4,...,Linearization C#94,Linearization C#95,Linearization C#96,Linearization C#97,Linearization C#98,Linearization C#99,"traveled total, m",No. of completed missions,No. of collisions,No. of near-misses
0,map-generator/generated-maps/2024-11-28_13:19:...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3910.4,4,0,0
1,map-generator/generated-maps/2024-11-28_13:19:...,0,0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8252.6,7,0,0
2,map-generator/generated-maps/2024-11-28_13:19:...,0,0,0,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5514.5,8,0,0
3,map-generator/generated-maps/2024-11-28_13:19:...,0,0,0,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9415.9,5,0,0
4,map-generator/generated-maps/2024-11-28_13:19:...,0,0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3910.4,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,map-generator/generated-maps/2024-11-28_13:19:...,0,0,1,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9165.6,6,0,0
1436,map-generator/generated-maps/2024-11-28_13:19:...,0,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6178.3,9,1,2
1437,map-generator/generated-maps/2024-11-28_13:19:...,0,1,1,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9177.4,10,0,2
1438,map-generator/generated-maps/2024-11-28_13:19:...,0,1,1,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9144.0,9,0,0


In [8]:
def split_df_to_X_y(df):
    columns_input_df = [col for col in df.columns
                        if col.split('#')[0] in COLUMNS_INPUT]
    columns_output_df = list(COLUMNS_OUTPUT)
    assert set(COLUMNS_NONSPLIT) | set(columns_input_df) | set(columns_output_df) == set(df.columns)
    
    X = df[columns_input_df]
    y = df[columns_output_df]
    return X, y


def run_regression(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    model = sklearn.linear_model.LinearRegression()
    model.fit(X_train, y_train)
    ndarray_predictions = model.predict(X_test)
    df_predictions = pd.DataFrame(ndarray_predictions, columns=y_test.columns)
    return df_predictions


df_predictions_regression = run_regression(df_train, df_test)
df_predictions_regression

,"traveled total, m",No. of completed missions,No. of collisions,No. of near-misses
0,8200.618662,9.702666,5.302953,0.689518
1,7122.290812,4.317306,4.415833,0.030768
2,6467.878294,9.055693,-0.847103,-0.699063
3,8015.317654,9.562449,0.025095,-0.385185
4,8290.341475,9.825543,4.769176,0.703584
...,...,...,...,...
1435,8817.628800,5.587069,-0.148499,-0.204783
1436,8303.159628,12.154781,3.326117,0.599862
1437,8782.795955,10.857288,-0.328054,0.117620
1438,9195.174844,9.485071,-1.326640,-0.040280


In [9]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_DATA}/{basename}-{random.random()}.png'
    fig.savefig(filename)
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    display(HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))
    
    plt.close(fig)
    
    return filename


def evaluate_and_plot_column(df_test, df_predictions, column):
    y_test_column = df_test[column]
    predictions_column = df_predictions[column]
    
    r2 = sklearn.metrics.r2_score(y_test_column, predictions_column)
    print(f"{column}:")
    print(f"- R^2 Score: {r2}")

    # Plot results for each output column
    fig = plt.figure(figsize=(10, 6))
    plt.scatter(y_test_column, predictions_column, color='blue', alpha=0.5)
    plt.plot([y_test_column.min(), y_test_column.max()], [y_test_column.min(), y_test_column.max()], 'k--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted Values for {column}')
    plt.grid(True)
    save_and_show(fig, f'Actual_vs_Predicted_Values_{column}')


def evaluate_and_plot_all_columns(df_test, df_predictions):
    for column in COLUMNS_OUTPUT:
        evaluate_and_plot_column(df_test, df_predictions, column)
        
        
evaluate_and_plot_all_columns(df_test, df_predictions_regression)

traveled total, m:
- R^2 Score: 0.3807744825804721


No. of completed missions:
- R^2 Score: 0.5264902565566945


No. of collisions:
- R^2 Score: 0.21391640787085497


No. of near-misses:
- R^2 Score: -0.5370411652677056


In [10]:
def run_autogluon(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    # Train AutoGluon models
    predictors = []
    df_predictions = pd.DataFrame()
    for column in COLUMNS_OUTPUT:
        print(f'{column=}:')
        df_train_predictor = pd.concat([X_train, y_train[[column]]], axis=1)
        
        predictor = autogluon.tabular.TabularPredictor(
            label=column, 
            eval_metric='r2', 
            problem_type='regression',
        ).fit(
            df_train_predictor,
            presets='medium',
            hyperparameters={
                'GBM': {},       # LightGBM (TODO: something like `GBMLarge`)
                'XGB': {},       # XGBoost
                'RF': {},        # Random Forest
                'XT': {},        # Extra Trees
                # 'CAT': {},      # CatBoost, omitted if slow
                # 'NN': {},       # Neural net, if you want it
                # 'LR': {},       # Linear model
                # 'KNN': {},      # K-Nearest Neighbors
            },
        )
        predictors.append(predictor)
        
        df_predictions[column] = predictor.predict(X_test)

        # Leaderboard - Display a table of different models and their performance
        df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        leaderboard = predictor.leaderboard(df_test_predictor, silent=True)
        show(leaderboard, f'Leaderboard for {column}')
        
        # Feature importance on training data
        # show(
        #     predictor.feature_importance(df_train_predictor),
        #     'feature_importance(df_train_predictor)'
        # )
        # 
        # # Feature importance on test data
        # show(
        #     predictor.feature_importance(df_test_predictor),
        #     'feature_importance(df_test_predictor)'
        # )
        # Example: SHAP values for a specific model
        # shap_values = predictor.get_model_shap_values(df_test_predictor, model='LightGBM')
        # show(shap_values, 'shap_values')  # SHAP values for each feature and each prediction
        
    return predictors, df_predictions


predictors, df_predictions_autogluon = run_autogluon(df_train, df_test)
df_predictions_autogluon

No path specified. Models will be saved in: "AutogluonModels/ag-20241220_153507"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #135~20.04.1-Ubuntu SMP Mon Oct 7 13:56:22 UTC 2024
CPU Count:          16
Memory Avail:       6.54 GB / 31.09 GB (21.0%)
Disk Space Avail:   314.20 GB / 693.60 GB (45.3%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20241220_153507"
Train Data Rows:    5736
Train Data Columns: 104
Label Column:       traveled total, m
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6683.95 MB
	Train Data (Origina

column='traveled total, m':


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 2 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 100 | ['Linearization C#0', 'Linearization C#1', 'Linearization C#2', 'Linearization C#3', 'Linearization C#4', ...]
		('int', [])   :   4 | ['Coordination strategy', 'isCanPassFirstHum', 'isRacingThroughCrossroadAllowed', 'Vehicle ID']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 100 | ['Linearization C#0', 'Linearization C#1', 'Linearization C#2', 'Linearization C#3', 'Linearization C#4', ...]
		('int', [])       :   2 | ['Coordination strategy', 'Vehicle ID']
		('int', ['bool']) : 

[1000]	valid_set's l2: 347665	valid_set's r2: 0.94342
[2000]	valid_set's l2: 323701	valid_set's r2: 0.94732


	0.9476	 = Validation score   (r2)
	7.57s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForest ...
	0.9566	 = Validation score   (r2)
	4.28s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTrees ...
	0.9594	 = Validation score   (r2)
	1.67s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: XGBoost ...
	0.9502	 = Validation score   (r2)
	9.91s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'ExtraTrees': 0.7, 'XGBoost': 0.3}
	0.9614	 = Validation score   (r2)
	0.05s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 25.71s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 9601.0 rows/s (574 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20241220_153507")


...

	0.6504	 = Validation score   (r2)
	2.01s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForest ...


[1000]	valid_set's l2: 0.281864	valid_set's r2: 0.64617


	0.6491	 = Validation score   (r2)
	3.91s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTrees ...
	0.6648	 = Validation score   (r2)
	2.69s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	0.6613	 = Validation score   (r2)
	4.24s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'ExtraTrees': 0.471, 'XGBoost': 0.412, 'LightGBM': 0.118}
	0.6753	 = Validation score   (r2)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 13.27s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 12523.1 rows/s (574 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20241220_153628")


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTrees,-0.634064,0.664817,r2,0.077084,0.033813,2.693810,0.077084,0.033813,2.693810,1,True,3
1,WeightedEnsemble_L2,-0.691048,0.675298,r2,0.152187,0.045835,9.031093,0.002334,0.001006,0.091429,2,True,5
2,XGBoost,-0.803467,0.661332,r2,0.022554,0.005053,4.238377,0.022554,0.005053,4.238377,1,True,4
3,LightGBM,-1.042679,0.650432,r2,0.050214,0.005963,2.007478,0.050214,0.005963,2.007478,1,True,1
4,RandomForest,-1.180127,0.649087,r2,0.152507,0.034634,3.906909,0.152507,0.034634,3.906909,1,True,2


,"traveled total, m",No. of completed missions,No. of collisions,No. of near-misses
0,5182.628906,6.920036,0.088024,-0.046428
1,8134.416992,7.474953,0.927449,0.003770
2,6122.679199,8.166824,-0.155424,0.005856
3,9235.293945,6.408742,1.123160,0.040558
4,5250.327148,7.155142,0.072012,0.002781
...,...,...,...,...
1435,9153.595703,6.034895,0.010371,0.008559
1436,5946.552246,8.723999,-0.133251,0.970391
1437,9064.640625,9.477354,-0.009502,0.910473
1438,8956.281250,9.097409,-0.011393,0.033908


In [11]:
evaluate_and_plot_all_columns(df_test, df_predictions_autogluon)

traveled total, m:
- R^2 Score: 0.8455460668569312


No. of completed missions:
- R^2 Score: 0.9023364782333374


No. of collisions:
- R^2 Score: 0.6627932786941528


No. of near-misses:
- R^2 Score: -0.6910476684570312


In [12]:
def explain_predictions(predictors, df_test):
    # X_test, y_test = split_df_to_X_y(df_test)
    
    for column, predictor in zip(COLUMNS_OUTPUT, predictors):
        # df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        
        tree_model = predictor._trainer.load_model('LightGBM')
        tree_importance = tree_model.model.feature_importance(importance_type='gain')
        # show(tree_importance, column)
        
        feature_names = predictor.feature_metadata.get_features()
        df = pd.DataFrame.from_dict(
            {name: {'importance': value} for name, value in zip(feature_names, tree_importance)},
            orient='index'
        )
        # print(df.index)
        # Group indexes by their base name before `#`
        df['group'] = df.index.str.extract(r'^(.+?)(?:#\d+)?$', expand=False)
        df = df.groupby('group')['importance'].sum().to_frame()
        
        df.sort_values(by='importance', ascending=False, inplace=True)
        show(df, column)
        
        
explain_predictions(predictors, df_test)

,importance
group,
Linearization C,2.486237e+11
Coordination strategy,5.175046e+10
Vehicle ID,3.087588e+10
isCanPassFirstHum,1.256243e+09
isRacingThroughCrossroadAllowed,4.582813e+08


,importance
group,
Linearization C,837631.333164
Coordination strategy,93680.758545
Vehicle ID,50492.565954
isCanPassFirstHum,2634.737267
isRacingThroughCrossroadAllowed,1058.428485


,importance
group,
Linearization C,301076.576980
Coordination strategy,121063.380209
Vehicle ID,107510.925374
isRacingThroughCrossroadAllowed,12720.081166
isCanPassFirstHum,1810.975779


,importance
group,
Linearization C,34877.544957
Coordination strategy,3336.126039
Vehicle ID,2188.727619
isRacingThroughCrossroadAllowed,642.932879
isCanPassFirstHum,221.631843
